In [1]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

# Subtract Biases and Darks

In [2]:
bias_dir = './data/bias/'
dark_dir = './data/dark/'
flat_dir = './data/flat/'

combined_bias = CCDData.read(bias_dir + 'combined_bias.fits')
combined_dark = CCDData.read(dark_dir + 'combined_dark_900.000.fits')
raw_V_flats = ImageFileCollection(flat_dir + 'V_raw')
raw_V_flats = raw_V_flats.files_filtered(
    include_path=True
)

for file_name in raw_V_flats:
    ccd = CCDData.read(file_name, unit='adu')
    processed = ccdp.subtract_bias(ccd, combined_bias)
    processed = ccdp.subtract_dark(
        processed,
        combined_dark,
        exposure_time='exptime',
        exposure_unit=u.second,
        scale=True
    )
    
    name = file_name.split('/')[-1]
    ccd.write(flat_dir + f'V_reduced/{name}', overwrite=True)

# Combine Flats

In [3]:
reduced_V_flats = ImageFileCollection(flat_dir + 'V_reduced')

def recip_median(a):
    return 1 / np.median(a)

combined_V_flat = ccdp.combine(
    reduced_V_flats.files_filtered(include_path=True),
    scale=recip_median,
    sigma_clip=True,
    sigma_clip_low_thresh=5,
    sigma_clip_high_thresh=5,
    sigma_clip_func=np.ma.median,
    sigma_clip_dev_func=mad_std,
    mem_limit=350e+6
)

combined_V_flat.meta['combined'] = True
combined_V_flat.write(flat_dir + 'combined_V_flat.fits', overwrite=True)

INFO:astropy:splitting each image into 4 chunks to limit memory usage to 350000000.0 bytes.


INFO: splitting each image into 4 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
